In [63]:
import pandas as pd
import numpy as np
from docplex.cp.model import  CpoModel

In [64]:
def extract_solution(msol, df, extract_dvar_names=None, drop_column_names=None, drop:bool=True):
    df = df.copy()
    """Generalized routine to extract a solution value. 
    Can remove the dvar column from the df to be able to have a clean df for export into scenario."""
    if extract_dvar_names is not None:
        for xDVarName in extract_dvar_names:
            if xDVarName in df.columns:
                df[f'{xDVarName}_Solution'] = [msol.get_value(dvar)   for dvar in df[xDVarName]]  #dvar.solution_value
                if drop:
                    df = df.drop([xDVarName], axis = 1)
    if drop and drop_column_names is not None:
        for column in drop_column_names:
            if column in df.columns:
                df = df.drop([column], axis = 1)
    return df    

![alternatvie text](2023-07-07-11-43-28.png)

https://developers.google.com/optimization/cp/queens

In [65]:
mdl = CpoModel(name="NQueens")

In [66]:
board_size = 8  # boardsize

# Every column must have a queen, we set the row number as DV
queens = mdl.integer_var_list(min = 0, max =board_size-1, size = board_size, name = "x" )
queens = pd.DataFrame(queens, columns = ["queen_row_number"])
queens['queen_column_number']=queens.index
queens

,queen_row_number,queen_column_number
0,x_0 = intVar(0..7),0
1,x_1 = intVar(0..7),1
2,x_2 = intVar(0..7),2
3,x_3 = intVar(0..7),3
4,x_4 = intVar(0..7),4
5,x_5 = intVar(0..7),5
6,x_6 = intVar(0..7),6
7,x_7 = intVar(0..7),7


In [67]:
# The row numbers of queens must be different

mdl.add( mdl.all_diff(queens['queen_row_number']) )

The diagonal constraint is a little trickier than the row and column constraints. First, if two queens lie on the same diagonal, one of the following conditions must be true:

The row number plus the column number for each of the two queens are equal. In other words, $queens(j) + j$ has the same value for two different indices $j$.

The row number minus the column number for each of the two queens are equal. In this case, $queens(j) - j$ has the same value for two different indices $j$.

One of these conditions means the queens lie on the same ascending diagonal ( going from left to right), while the other means they lie on the same descending diagonal. Which condition corresponds to ascending and which to descending depends on how you order the rows and columns.

So the diagonal constraint is that the values of $queens(j) + j$ must all be different, and the values of $queens(j) - j$ must all be different, for different $j$.

In [68]:
queens['U']= queens.apply(lambda x:x['queen_row_number']+x['queen_column_number'], axis=1)
queens['V']= queens.apply(lambda x:x['queen_row_number']-x['queen_column_number'], axis=1)

display(queens)

mdl.add( mdl.all_diff(queens['U']) )
mdl.add( mdl.all_diff(queens['V']) )

,queen_row_number,queen_column_number,U,V
0,x_0 = intVar(0..7),0,x_0 + 0,x_0 - 0
1,x_1 = intVar(0..7),1,x_1 + 1,x_1 - 1
2,x_2 = intVar(0..7),2,x_2 + 2,x_2 - 2
3,x_3 = intVar(0..7),3,x_3 + 3,x_3 - 3
4,x_4 = intVar(0..7),4,x_4 + 4,x_4 - 4
5,x_5 = intVar(0..7),5,x_5 + 5,x_5 - 5
6,x_6 = intVar(0..7),6,x_6 + 6,x_6 - 6
7,x_7 = intVar(0..7),7,x_7 + 7,x_7 - 7


In [69]:
msol = mdl.solve(TimeLimit=10, log_output = None)
#msol.print_solution()
queens_solution = extract_solution(msol, queens,extract_dvar_names=['queen_row_number'])
queens_solution

,queen_column_number,U,V,queen_row_number_Solution
0,0,x_0 + 0,x_0 - 0,4
1,1,x_1 + 1,x_1 - 1,1
2,2,x_2 + 2,x_2 - 2,7
3,3,x_3 + 3,x_3 - 3,0
4,4,x_4 + 4,x_4 - 4,3
5,5,x_5 + 5,x_5 - 5,6
6,6,x_6 + 6,x_6 - 6,2
7,7,x_7 + 7,x_7 - 7,5


In [70]:
for i in range(0,board_size):
    for j in range(0,board_size):
            m = queens_solution[(queens_solution['queen_row_number_Solution']==i) & (queens_solution['queen_column_number']==j)  ]
            if len(m) ==0:
                  print('_', end=' ')
            else:
                  print('Q', end=' ')
    print()
print()

_ _ _ Q _ _ _ _ 
_ Q _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ _ _ Q _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ Q 
_ _ _ _ _ Q _ _ 
_ _ Q _ _ _ _ _ 

